### 左右反転、閾値処理、ぼかしで８倍の水増し

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
def scratch_image(img, flip=True, thr=True, fit=True):
    ## 水増しの手法を配列にする
    methods = [flip, thr, fit]
    
    ## ぼかしのフィルター作成
    filter1 = np.ones((3,3))
    images = [img]
    
    ## 水増しの手法
    scratch = np.array([
        lambda x: cv2.flip(x, 1),
        lambda x: cv2.threshold(x, 100, 255, cv2.THRESH_TOZERO)[1],
        lambda x: cv2.GaussianBlur(x, (5, 5), 0)
    ])
    
    doubling_images = lambda f, imag: np.r_[imag, [f(i) for i in imag]] 
    
    for func in scratch[methods]:
        images = doubling_images(func, images)
    return images

In [3]:
import glob

in_dir = "face_image/*"
## in_dirからのファイル名取得(ディレクトリ名含む)
in_jpg = glob.glob(in_dir)
## 画像ファイル名のみ
img_file_name_list = os.listdir("face_image/")

for i in range(len(in_jpg)):
    img = cv2.imread(str(in_jpg[i]))
    scratch_face_images = scratch_image(img)

    for num, im in enumerate(scratch_face_images):
            fn, ext = os.path.splitext(img_file_name_list[i])
            file_name=os.path.join("./face_scratch_image",str(fn+"."+str(num)+".jpg"))
            cv2.imwrite(str(file_name) ,im)

### 画像と正解ラベルをリストにする

In [28]:
import random
from keras.utils.np_utils import to_categorical
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [65]:
img_file_name_list = os.listdir("masami_train/")
# print(len(img_file_name_list))

for i in range(len(img_file_name_list)):
    n = os.path.join("masami_train", img_file_name_list[i-1])
    img = cv2.imread(n)
    
    if isinstance(img, type(None)) == True:
        img_file_name_list.pop(i)
        continue

In [66]:
X_train=[]
y_train=[]

for j in range(0, len(img_file_name_list)-1):
    n = os.path.join("masami_train/",img_file_name_list[j])
    img = cv2.imread(n)
    b,g,r = cv2.split(img)
    img = cv2.merge([r,g,b])
    X_train.append(img)
    n = img_file_name_list[j]
    y_train = np.append(y_train, n[0:2]).reshape(j+1,1)    
X_train = np.array(X_train)

img_file_name_list=os.listdir("test_image/")
# print(len(img_file_name_list))

In [67]:
for i in range(len(img_file_name_list)-1):
    n=os.path.join("test_image/",img_file_name_list[i-1])
    img = cv2.imread(n)
    if isinstance(img,type(None)) == True:
        img_file_name_list.pop(i)
        continue

12


In [72]:
X_test=[]
y_test=[]

for j in range(0, len(img_file_name_list)):
    n=os.path.join("test_image/", img_file_name_list[j])
    img = cv2.imread(n)
    b,g,r = cv2.split(img)
    img = cv2.merge([r,g,b])
    X_test.append(img)
    n=img_file_name_list[j]
    y_test=np.append(y_test, n[0:2]).reshape(j+1,1)
X_test=np.array(X_test)